In [2]:
from numpy import dot
from numpy.linalg import norm
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
data = pd.read_excel('../Data/ИтоговыеДанные.xlsx')

In [4]:
# Группировка данных по клиентам и подсчет общего количества посещений и общего расхода
client_data = data.groupby('ID Клиента').agg({'МАГАЗИН': 'count', 'Сумма расходов': 'sum'})

# Создание вектора для каждого клиента
vectors = []
for client_id, row in client_data.iterrows():
    visits = row['МАГАЗИН']
    spending = row['Сумма расходов']
    vector = [visits, spending]
    vectors.append(vector)

# Преобразование векторов в DataFrame
cluster_vector = pd.DataFrame(vectors, columns=['Посещения МАГАЗИНА', 'Общий расход'], index=client_data.index)

In [5]:
cluster_vector

,Посещения МАГАЗИНА,Общий расход
ID Клиента,,
10157447,107,632039
10157454,1,163
10158439,27,635690
10158854,528,15423642
10159585,67,222196
...,...,...
11328050,186,8064333
11328121,13,1653598
11335636,1,884311


In [9]:
# Расчет Q1, Q2, Q3 для столбца "Посещения МАГАЗИНА"
mean_visits = cluster_vector['Посещения МАГАЗИНА'].mean()
q1_visits = cluster_vector['Посещения МАГАЗИНА'].quantile(0.25)
q2_visits = cluster_vector['Посещения МАГАЗИНА'].quantile(0.5)
q3_visits = cluster_vector['Посещения МАГАЗИНА'].quantile(0.75)

# Расчет Q1, Q2, Q3 для столбца "Общий расход"
mean_spending = cluster_vector['Общий расход'].mean()
q1_spending = cluster_vector['Общий расход'].quantile(0.25)
q2_spending = cluster_vector['Общий расход'].quantile(0.5)
q3_spending = cluster_vector['Общий расход'].quantile(0.75)

# Вывод результатов
print(f"mean: {mean_visits}")
print(f"Значения Q1, Q2, Q3 для столбца 'Посещения МАГАЗИНА':")
print(f"Q1: {q1_visits}")
print(f"Q2: {q2_visits}")
print(f"Q3: {q3_visits}")
print()
print(f"mean: {mean_spending}")
print(f"Значения Q1, Q2, Q3 для столбца 'Общий расход':")
print(f"Q1: {q1_spending}")
print(f"Q2: {q2_spending}")
print(f"Q3: {q3_spending}")


mean: 91.23822341857336
Значения Q1, Q2, Q3 для столбца 'Посещения МАГАЗИНА':
Q1: 3.0
Q2: 15.0
Q3: 67.0

mean: 13973332.67294751
Значения Q1, Q2, Q3 для столбца 'Общий расход':
Q1: 14901.5
Q2: 191154.0
Q3: 1639363.0


In [23]:
# Определение группы на основе суммы расходов и посещений МАГАЗИНОВ
def get_group(row):
    if row['Общий расход'] <= q2_spending and row['Посещения МАГАЗИНА'] <= mean_visits:
        return 'Низкий расход, низкие посещения'
    elif row['Общий расход'] <= q2_spending and row['Посещения МАГАЗИНА'] > mean_visits:
        return 'Низкий расход, высокие посещения'
    elif row['Общий расход'] > q2_spending and row['Посещения МАГАЗИНА'] <= mean_visits:
        return 'Высокий расход, низкие посещения'
    else:
        return 'Высокий расход, высокие посещения'

# Добавление столбца с группой в DataFrame
cluster_vector['Группа'] = cluster_vector.apply(get_group, axis=1)

In [24]:
cluster_vector

,Посещения МАГАЗИНА,Общий расход,Группа
ID Клиента,,,
10157447,107,632039,"Высокий расход, высокие посещения"
10157454,1,163,"Низкий расход, низкие посещения"
10158439,27,635690,"Высокий расход, низкие посещения"
10158854,528,15423642,"Высокий расход, высокие посещения"
10159585,67,222196,"Высокий расход, низкие посещения"
...,...,...,...
11328050,186,8064333,"Высокий расход, высокие посещения"
11328121,13,1653598,"Высокий расход, низкие посещения"
11335636,1,884311,"Высокий расход, низкие посещения"


In [25]:
# Создание словаря для хранения списков клиентов по группам
grouped_clients = {}

# Итерация по каждой группе и добавление клиентов в соответствующий список
for group in cluster_vector['Группа'].unique():
    clients = cluster_vector[cluster_vector['Группа'] == group].index.tolist()
    grouped_clients[group] = clients

# Вывод списка клиентов для каждой группы
for group, clients in grouped_clients.items():
    print(f"Группа: {group}")
    print(f"ID клиентов: {clients}")
    print()

Группа: Высокий расход, высокие посещения
ID клиентов: [10157447, 10158854, 10161672, 10205250, 10210111, 10212548, 10213259, 10213348, 10213902, 10214586, 10215398, 10215689, 10216133, 10216655, 10216806, 10216816, 10217384, 10218204, 10218209, 10218604, 10218864, 10219502, 10220154, 10220163, 10220687, 10220770, 10354174, 10354725, 10355392, 10355417, 10355445, 10355641, 10356174, 10356763, 10358256, 10359664, 10360759, 10360800, 10361502, 10361741, 10363026, 10371447, 10371835, 10372259, 10372464, 10402701, 10404555, 10404735, 10406690, 10407379, 10409000, 10489533, 10495894, 10544114, 10545378, 10562323, 10562352, 10562428, 10565113, 10567166, 10567954, 10574034, 10574664, 10574677, 10574795, 10577475, 10578913, 10579558, 10579621, 10580196, 10580277, 10580935, 10583558, 10583563, 10687823, 10689235, 10718802, 10806009, 10830303, 10831118, 10834321, 10834988, 10836387, 10837346, 10837991, 10867515, 10887712, 10989669, 10996693, 11000774, 11002153, 11004905, 11007017, 11008961, 1101